# Capacity Expansion Planning with EV Inclusion 

(Title will be revised in Final Version)

_Written by Kaarthi A Gnapathy for MAE 243 - Electric Power Systems Modelling Course, Fall 23_

### Loading Packages

In [3]:
using JuMP
using HiGHS
using Plots
using DataFrames, CSV
using Clp
using GLPK

### Loading Inputs (1)

In [1]:
#working directory **CHANGE IF WORKING FROM A DIFFERENT DIRECTORY/COMPUTER

#input path for reference electrification
inputs_path_ref = "complex_expansion_data/10_days/"

#input path for high electrification
#inputs_path_high = "wecc_2045_all_clean_expansion/high_electrification_16_weeks/"

"complex_expansion_data/10_days/"

In [26]:
#similar generator info will be used since only demand profiles are different between both datasets
generators = DataFrame(CSV.File(joinpath(inputs_path_ref, "Generators_data.csv")));

#ID for all generators for easy reference
G = generators.R_ID;

#generators

In [4]:
#reading reference demand input data
demand_inputs_ref = DataFrame(CSV.File(joinpath(inputs_path_ref, "Load_data.csv")))

#value of losr load (cost of involuntary non-served energy)
VOLL = demand_inputs_ref.Voll[1]

#set of price responsive demand (non-served energy) segments
S = convert(Array{Int64}, collect(skipmissing(demand_inputs_ref.Demand_segment)))

#creating a data frame for nse segments
nse = DataFrame(Segment = S, 
                NSE_Cost = VOLL.*collect(skipmissing(demand_inputs_ref.Cost_of_demand_curtailment_perMW)),
                NSE_Max = collect(skipmissing(demand_inputs_ref.Max_demand_curtailment)))

#set of sequential hours per sub-period
hours_per_period = convert(Int64, demand_inputs_ref.Hours_per_period[1])
#set of time sample sub-periods
P = convert(Array{Int64}, 1:demand_inputs_ref.Subperiods[1])
#sub-period cluster weights = number of hours represented by each sample period
W = convert(Array{Int64}, collect(skipmissing(demand_inputs_ref.Sub_Weights)))
#set of all time steps
T = convert(Array{Int64}, demand_inputs_ref.Time_index)

#creating a vector of sample weights
sample_weight = zeros(Float64, size(T,1))
t=1
for p in P
    for h in 1:hours_per_period
        sample_weight[t]=W[p]/hours_per_period
        t += 1
    end
end

#set of zones
Z = convert(Array{Int64}, 1:3)

demand = select(demand_inputs_ref, :Load_MW_z1, :Load_MW_z2, :Load_MW_z3);

#demand_ref

In [5]:
#read generator capacity factors by hour
variability = DataFrame(CSV.File(joinpath(inputs_path_ref, "Generators_variability.csv")))
#cleaning up the data by removing the first column of indexes
variability = variability[:,2:ncol(variability)];

#variability

In [6]:
#read fuels data
fuels = DataFrame(CSV.File(joinpath(inputs_path_ref, "Fuels_data.csv")));

#fuels

In [7]:
#reading network data
lines = DataFrame(CSV.File(joinpath(inputs_path_ref, "Network.csv")));
#removing the name of the line connections
lines = select(lines[1:2,:], 
    :Network_lines, :z1, :z2, :z3, 
    :Line_Max_Flow_MW, :Line_Min_Flow_MW, :Line_Loss_Percentage, 
    :Line_Max_Reinforcement_MW, :Line_Reinforcement_Cost_per_MW_yr)
#fixed O&M costs for lines
lines.Line_Fixed_Cost_per_MW_yr = lines.Line_Reinforcement_Cost_per_MW_yr./20
#set of all lines

L = convert(Array{Int64}, lines.Network_lines);

#lines

In [8]:
#setting zones for easy reference
zones = ["z1", "z2", "z3"];

In [11]:
#calculating the associated variable costs for generators

generators.Var_Cost = zeros(Float64, size(G,1))
generators.CO2_Rate = zeros(Float64, size(G,1))
generators.Start_Cost = zeros(Float64, size(G,1))
generators.CO2_Per_Start = zeros(Float64, size(G,1))

for g in G
    # Variable cost ($/MWh) = variable O&M ($/MWh) + fuel cost ($/MMBtu) * heat rate (MMBtu/MWh)
    generators.Var_Cost[g] = generators.Var_OM_cost_per_MWh[g] +
        fuels[fuels.Fuel.==generators.Fuel[g],:Cost_per_MMBtu][1]*generators.Heat_rate_MMBTU_per_MWh[g]
    # CO2 emissions rate (tCO2/MWh) = fuel CO2 content (tCO2/MMBtu) * heat rate (MMBtu/MWh)
    generators.CO2_Rate[g] = fuels[fuels.Fuel.==generators.Fuel[g],:CO2_content_tons_per_MMBtu][1]*generators.Heat_rate_MMBTU_per_MWh[g]
    # Start-up cost ($/start/MW) = start up O&M cost ($/start/MW) + fuel cost ($/MMBtu) * start up fuel use (MMBtu/start/MW) 
    generators.Start_Cost[g] = generators.Start_cost_per_MW[g] +
        fuels[fuels.Fuel.==generators.Fuel[g],:Cost_per_MMBtu][1]*generators.Start_fuel_MMBTU_per_MW[g]
    # Start-up CO2 emissions (tCO2/start/MW) = fuel CO2 content (tCO2/MMBtu) * start up fuel use (MMBtu/start/MW) 
    generators.CO2_Per_Start[g] = fuels[fuels.Fuel.==generators.Fuel[g],:CO2_content_tons_per_MMBtu][1]*generators.Start_fuel_MMBTU_per_MW[g]
end


In [30]:
#Subset definition

#subset of thermal generators that are subject to unit commitment constraints
UC = intersect(generators.R_ID[generators.Commit.==1], G)

#subset of generators that are not subject to unit commitment constraints
ED = intersect(generators.R_ID[.!(generators.Commit.==1)], G)

#subset of storage resources
STOR = intersect(generators.R_ID[generators.STOR.>=1], G)

#subset of variable renewable resources
VRE = intersect(generators.R_ID[generators.DISP.==1], G)

#subset of new build generators
NEW = intersect(generators.R_ID[generators.New_Build.==1], G)

#subset of existing generators
OLD = intersect(generators.R_ID[.!(generators.New_Build.==1)], G)

#subset of RPS qualifying resources
RPS = intersect(generators.R_ID[generators.RPS.==1], G);

#subset of time steps that begin a sub period
START = 1:hours_per_period:maximum(T)

#subset of time periods that do not begin a sub period
INTERIOR = setdiff(T, START);

### Defining Model Function (2)

In [19]:
#creating a function for the capacity expansion model to test out different scenarios

function capacity_expansion(demand_profile, optimizer = HiGHS.Optimizer)
    CE = Model(optimizer)


    #DECISION VARIABLES

    #Capacity decision variables
    @variables(CE, begin

            #standard capacity variables
            vCAP[g in G]               >= 0 # power capacity (MW)
            vRET_CAP[g in OLD]         >= 0 # retirement power capacity (MW)
            vNEW_CAP[g in NEW]         >= 0 # new build power capacity (MW)
            
            #storage variables
            vE_CAP[g in STOR]          >= 0 # storage energy capacity (MWh)
            vRET_E_CAP[g in intersect(STOR, OLD)]   >= 0 # retirement storage energy capacity
            vNEW_E_CAP[g in intersect(STOR, NEW)]   >= 0 # new build storage energy capacity

            #transmission variables
            vT_CAP[l in L]             >= 0 # transmission capacity (MW)
            vRET_T_CAP[l in L]         >= 0 # retirement transmission capacity (MW)
            vNEW_T_CAP[l in L]         >= 0 # new build transmission capacity (MW)
    end)

    #unbounded till max_cap data is received
    #for g in NEW[generators[NEW, :Max_Cap_MW].>0]
    #    set_upper_bound(vNew_CAP[g], generators.Max_Cap_MW[g])
    #end

    #set upper bounds on transmission capacity expansion
    for l in L
        set_upper_bound(vNEW_T_CAP[l], lines.Line_Max_Reinforcement_MW[l])
    end

    #operational decision variables
    @variables(CE, begin
            vGEN[T, G]          >= 0 # power generation (MW)
            vCHARGE[T, STOR]    >= 0 # power charging (MW)
            vSOC[T, STOR]       >= 0 # energy storage state of charge (MW)
            vNSE[T, S, Z]       >= 0 # non-served energy/demand curtailment (MW)
            vFLOW[T, L]         >= 0 # transmission line flow (MW)
    end)

    #CONSTRAINTS

    #Supply Demand Balance Constraint
    @constraint(CE, cDemandBalance[t in T, z in Z], 
        sum(vGEN[t,g] for g in intersect(generators[generators.zone.==z,:R_ID],G)) +
        sum(vNSE[t,s,z] for s in S) - 
        sum(vCHARGE[t,g] for g in intersect(generators[generators.zone.==z,:R_ID],STOR)) -
        demand_profile[t,z] - 
        sum(lines[l,Symbol(string("z",z))] * vFLOW[t,l] for l in L) == 0
        )

    #capacitated constraints
    @constraints(CE, begin

            #max power constraint
            cMaxPower[t in T, g in G], vGEN[t,g] <= variability[t,g]*vCAP[g]
            #max charge constraint
            cMaxCharge[t in T, g in STOR], vCHARGE[t,g] <= vCAP[g]
            #max state of charge constraint
            cMaxSOC[t in T, g in STOR], vSOC[t,g] <= vE_CAP[g]
            #max NSE constraint
            cMaxNSE[t in T, s in S, z in Z], vNSE[t,s,z] <= nse.NSE_Max[s]*demand_profile[t,z]
            #max flow constraint
            cMaxFlow[t in T, l in L], vFLOW[t,l] <= vT_CAP[l]
            #min flow constraint
            cMinFlow[t in T, l in L], vFLOW[t,l] >= -vT_CAP[l]        
        end)

    #total capacity constraint
    @constraints(CE, begin

            #total capacity for exisiting units
            cCapOld[g in OLD], vCAP[g] == generators.Existing_Cap_MW[g] - vRET_CAP[g]

            #total capacity for new units
            cCapNew[g in NEW], vCAP[g] == vNEW_CAP[g]

            #total energy storage capacity for existing units
            cCapEnergyOld[g in intersect(STOR, OLD)], 
                vE_CAP[g] == generators.Existing_Cap_MWh[g] - vRET_E_CAP[g]

            #total energy storage capacity for new units
            cCapEnergyNew[g in intersect(STOR, NEW)], 
                vE_CAP[g] == vNEW_E_CAP[g]

            #total transmission capacity
            cTransCap[l in L], vT_CAP[l] == lines.Line_Max_Flow_MW[l] - vRET_T_CAP[l] + vNEW_T_CAP[l]            
        end)

    # time coupling constraints
    @constraints(CE, begin

            #ramp up, normal
            cRampUp[t in INTERIOR, g in G],
                vGEN[t,g] - vGEN[t-1,g] <= generators.Ramp_Up_percentage[g]*vCAP[g]

            #ramp up, sub-period wrapping
            cRampUpWrap[t in START, g in G],
                vGEN[t,g] - vGEN[t+hours_per_period-1,g] <= generators.Ramp_Up_percentage[g]*vCAP[g]

            #ramp down, normal
            cRampDown[t in INTERIOR, g in G],
                vGEN[t-1,g] - vGEN[t,g] <= generators.Ramp_Dn_percentage[g]*vCAP[g]

            #ramp down, sub-period warping
            cRampDownWrap[t in START, g in G],
                vGEN[t+hours_per_period-1,g] - vGEN[t,g] <= generators.Ramp_Dn_percentage[g]*vCAP[g]

            #storage state of charge, normal
            cSOC[t in INTERIOR, g in STOR],
                vSOC[t,g] == vSOC[t-1,g] + generators.Eff_up[g]*vCHARGE[t,g] - vGEN[t,g]/generators.Eff_down[g]

            #storage state of charge, wrapping
            cSOCWrap[t in START, g in STOR], 
                vSOC[t,g] == vSOC[t+hours_per_period-1,g] + generators.Eff_up[g]*vCHARGE[t,g] - vGEN[t,g]/generators.Eff_down[g]
        end)

    #OBJECTIVE FUNCTION
    
    #fixed cost for generation
    @expression(CE, eFixedCostsGeneration,
        #fixed costs for total capacity
        sum(generators.Fixed_OM_cost_per_MWyr[g]*vCAP[g] for g in G) +
        #investment costs for new capacity
        sum(generators.Inv_cost_per_MWyr[g]*vNEW_CAP[g] for g in NEW)
        )
    
    #fixed cost for storage
    @expression(CE, eFixedCostsStorage,
        #fixed costs for total storage capacity
        sum(generators.Fixed_OM_cost_per_MWhyr[g]*vE_CAP[g] for g in STOR) + 
        #investment costs for new storage energy capacity
        sum(generators.Inv_cost_per_MWhyr[g]*vNEW_E_CAP[g] for g in intersect(STOR, NEW))
        )
    
    #fixed cost for transmission
    @expression(CE, eFixedCostsTransmission,
     # Investment and fixed O&M costs for transmission lines
        sum(lines.Line_Fixed_Cost_per_MW_yr[l]*vT_CAP[l] +
            lines.Line_Reinforcement_Cost_per_MW_yr[l]*vNEW_T_CAP[l] for l in L)
        )

    #variable costs for all generators
    @expression(CE, eVariableCosts,
     # Variable costs for generation, weighted by hourly sample weight 
    sum(sample_weight[t]*generators.Var_Cost[g]*vGEN[t,g] for t in T, g in G)
        )

    #NSE costs
    @expression(CE, eNSECosts,
     # Non-served energy costs, weighted by hourly sample weight to ensure non-served energy costs estimate annual costs
    sum(sample_weight[t]*nse.NSE_Cost[s]*vNSE[t,s,z] for t in T, s in S, z in Z)
        )

    @objective(CE, Min,
    eFixedCostsGeneration + eFixedCostsStorage + eFixedCostsTransmission +
    eVariableCosts + eNSECosts
        )
    optimize!(CE)

    return (
        CAP = vCAP,
        GEN = vGEN,
        E_CAP = vE_CAP,
        T_CAP = vT_CAP,
        NSE = vNSE,
        FixedCostsGeneration = eFixedCostsGeneration,
        FixedCostsStorage = eFixedCostsStorage,
        FixedCostsTransmission = eFixedCostsTransmission,
        VariableCosts = eVariableCosts,
        NSECosts = eNSECosts,
        cost = objective_value(CE)
        )
end


capacity_expansion (generic function with 2 methods)

### Solving the model (3)

In [21]:
@time solution = capacity_expansion(demand);

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
39122 rows, 14864 cols, 116392 nonzeros
39118 rows, 14864 cols, 116384 nonzeros
Presolve : Reductions: rows 39118(-5822); columns 14864(-1818); elements 116384(-13498)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 482(3.17783e+06) 0s
      11369     1.4606335766e+10 Pr: 0(0); Du: 0(1.35865e-09) 1s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 11369
Objective value     :  1.4606335766e+10
HiGHS run time      :          1.75
  2.231167 seconds (2.55 M allocations: 163.162 MiB, 5.86% gc time)


### Extracting results (4)

In [23]:
# Record generation capacity and energy results
generation = zeros(size(G,1))
for i in 1:size(G,1) 
    generation[i] = sum(sample_weight.*value.(solution.GEN)[:,G[i]].data) 
end


total_demand = sum(sum.(eachcol(sample_weight.*demand)))

peak_demand = maximum(sum(eachcol(demand)))
MWh_share = generation./total_demand.*100
cap_share = value.(solution.CAP).data./peak_demand.*100

generator_results = DataFrame(
    ID = G, 
    Resource = generators.Resource[G],
    Zone = generators.zone[G],
    Total_MW = value.(solution.CAP).data,
    Start_MW = generators.Existing_Cap_MW[G],
    Change_in_MW = value.(solution.CAP).data.-generators.Existing_Cap_MW[G],
    Percent_MW = cap_share,
    GWh = generation/1000,
    Percent_GWh = MWh_share
)

# energy storage energy capacity results (MWh)
storage_results = DataFrame(
    ID = STOR, 
    Zone = generators.zone[STOR],
    Resource = generators.Resource[STOR],
    Total_Storage_MWh = value.(solution.E_CAP).data,
    Start_Storage_MWh = generators.Existing_Cap_MWh[STOR],
    Change_in_Storage_MWh = value.(solution.E_CAP).data.-generators.Existing_Cap_MWh[STOR],
)


# transmission capacity results
transmission_results = DataFrame(
    Line = L, 
    Total_Transfer_Capacity = value.(solution.T_CAP).data,
    Start_Transfer_Capacity = lines.Line_Max_Flow_MW,
    Change_in_Transfer_Capacity = value.(solution.T_CAP).data.-lines.Line_Max_Flow_MW,
)


# non-served energy results by segment and zone
num_segments = maximum(S)
num_zones = maximum(Z)
nse_results = DataFrame(
    Segment = zeros(num_segments*num_zones),
    Zone = zeros(num_segments*num_zones),
    NSE_Price = zeros(num_segments*num_zones),
    Max_NSE_MW = zeros(num_segments*num_zones),
    Total_NSE_MWh = zeros(num_segments*num_zones),
    NSE_Percent_of_Demand = zeros(num_segments*num_zones)
)
i=1
for s in S
    for z in Z
        nse_results.Segment[i]=s
        nse_results.Zone[i]=z
        nse_results.NSE_Price[i]=nse.NSE_Cost[s]
        nse_results.Max_NSE_MW[i]=maximum(value.(solution.NSE)[:,s,z].data)
        nse_results.Total_NSE_MWh[i]=sum(sample_weight.*value.(solution.NSE)[:,s,z].data)
        nse_results.NSE_Percent_of_Demand[i]=sum(sample_weight.*value.(solution.NSE)[:,s,z].data)/total_demand*100
        i=i+1
    end
end

#cost_results (recorded in million dollars)
cost_results = DataFrame(
    Total_Costs = solution.cost/10^6,
    Fixed_Costs_Generation = value.(solution.FixedCostsGeneration)/10^6,
    Fixed_Costs_Storage = value.(solution.FixedCostsStorage)/10^6,
    Fixed_Costs_Transmission = value.(solution.FixedCostsTransmission)/10^6,
    Variable_Costs = value.(solution.VariableCosts)/10^6,
    NSE_Costs = value.(solution.NSECosts)/10^6
);

In [24]:
generator_results

Row,ID,Resource,Zone,Total_MW,Start_MW,Change_in_MW,Percent_MW,GWh,Percent_GWh
,Int64,String,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,onshore_wind_turbine,1,6751.84,8495.9,-1744.06,7.21482,14638.2,3.21055
2,2,biomass,2,0.0,48.4,-48.4,0.0,0.0,0.0
3,3,conventional_hydroelectric,2,0.0,467.02,-467.02,0.0,0.0,0.0
4,4,conventional_steam_coal,2,13567.7,13567.7,0.0,14.498,1.17736e5,25.8227
5,5,natural_gas_fired_combined_cycle,2,35592.6,36734.2,-1141.6,38.0332,1.10697e5,24.2787
6,6,natural_gas_fired_combustion_turbine,2,4704.48,4704.48,0.0,5.02707,76.212,0.0167153
7,7,natural_gas_steam_turbine,2,0.0,1118.5,-1118.5,0.0,0.0,0.0
8,8,nuclear,2,5020.0,5020.0,0.0,5.36422,43975.2,9.64492
9,9,onshore_wind_turbine,2,6979.0,6979.0,0.0,7.45755,18093.0,3.96827


In [25]:
storage_results

Row,ID,Zone,Resource,Total_Storage_MWh,Start_Storage_MWh,Change_in_Storage_MWh
,Int64,Int64,String,Float64,Float64,Float64
1,27,1,battery,-0.0,0.0,-0.0
2,40,2,battery,-0.0,0.0,-0.0
3,54,3,battery,-0.0,0.0,-0.0


In [27]:
transmission_results

Row,Line,Total_Transfer_Capacity,Start_Transfer_Capacity,Change_in_Transfer_Capacity
,Int64,Float64,Float64?,Float64
1,1,3271.05,3702.0,-430.954
2,2,1451.34,5529.0,-4077.66


In [28]:
nse_results

Row,Segment,Zone,NSE_Price,Max_NSE_MW,Total_NSE_MWh,NSE_Percent_of_Demand
,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,9000.0,0.0,0.0,0.0
2,1.0,2.0,9000.0,0.0,0.0,0.0
3,1.0,3.0,9000.0,0.0,0.0,0.0
4,2.0,1.0,603.0,0.0,0.0,0.0
5,2.0,2.0,603.0,6606.45,49326.3,0.0108186
6,2.0,3.0,603.0,407.1,1193.2,0.0002617


In [29]:
cost_results

Row,Total_Costs,Fixed_Costs_Generation,Fixed_Costs_Storage,Fixed_Costs_Transmission,Variable_Costs,NSE_Costs
,Float64,Float64,Float64,Float64,Float64,Float64
1,14606.3,4529.87,0.0,5.44231,10040.6,30.4633
